In [ ]:
import yfinance as yf
import pandas as pd
import time
import findspark
import os

In [ ]:
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("AutoLoader") \
    .getOrCreate()

In [ ]:
print(spark.version)

In [ ]:
hdfs_path = "hdfs://localhost:9000/project/stream/"
static_parquet_file = os.path.join(hdfs_path,"20250529225328.parquet")
print(static_parquet_file)
static_df = spark.read.parquet(static_parquet_file)
static_df.printSchema()
static_df.show()
schema = static_df.schema

In [ ]:
# parquet_file_path = os.path.join(os.getcwd(),"real time stock data")
# Use standard file source to read new Parquet files
hdfs_path = "hdfs://localhost:9000/project/stream/"
checkpoint_path = "hdfs://localhost:9000/project/checkpoint/"
destination_path = "hdfs://localhost:9000/project/output/"
df = spark.readStream \
    .schema(schema) \
    .format("parquet") \
    .load(hdfs_path)

# Process the streaming DataFrame (e.g., write to output for visualization)
query = df.writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", destination_path) \
    .option("checkpointLocation", checkpoint_path) \
    .start()

# Await termination of the query
query.awaitTermination()